# Numerical expression inference problem: the GEP-RNC algorithm
In this example, we aim to identify a mathematicalfunction $f$ from the given input-ouput data such that the function $f$ can produce the expected ouput given a certain input. This is a typical [symbolic regression](https://en.wikipedia.org/wiki/Symbolic_regression) problem:

> Symbolic regression is a type of regression analysis that searches the space of mathematical expressions to find the model that best fits a given dataset, both in terms of accuracy and simplicity.

which is most commonly solved with genetic programming and its variant, gene expression programming (GEP), presented here. 

Before continuing this example, you'd better first go through the two fundamental tutorials listed below to get familiar with GEP and *geppy*:
+ [Introduction to gene expression programming](https://geppy.readthedocs.io/en/latest/intro_GEP.html)
+ [Overview of geppy for Gene Expression Programming (GEP)](https://geppy.readthedocs.io/en/latest/overview.html)

The main difference of this problem w.r.t the [Boolean function identification](Boolean_function_identification.ipynb) problem is that generally a mathematical model involves constant coefficients, which is more challenging for both genetic programming (GP) and GEP. The standard way to handle numerical constants in GEP is to add another Dc domain in the genes dedicated to random numerical constant (RNC) evolution, i.e., the GEP-RNC algorithm. Please check Chapter 5 of *Ferreira, Cândida. Gene expression programming: mathematical modeling by an artificial intelligence. Vol. 21. Springer, 2006.* to learn more about the theory.

Alternatively, if the involved numerical constants are just simple integers, then the traditional ephemeral numerical constant (ENC) method can also be used, though it is not recommended in GEP. See the example [ENC for numerical expression inference](./numerical_expression_inference-ENC.ipynb).

To check the detailed documentation of each function/class in *geppy*, please refer to [library reference](https://geppy.readthedocs.io/en/latest/#library-reference).

In [1]:
import geppy as gep
from deap import creator, base, tools
import numpy as np
import random
import operator
import sympy as sp
import math

# for reproduction
s = 0
random.seed(s)
np.random.seed(s)

sym_map = {
    operator.and_.__name__: sp.And,
    operator.or_.__name__: sp.Or,
    operator.not_.__name__: sp.Not,
    operator.add.__name__: operator.add,
    operator.sub.__name__: operator.sub,
    operator.mul.__name__: operator.mul,
    operator.neg.__name__: operator.neg,
    operator.pow.__name__: operator.pow,
    operator.abs.__name__: operator.abs,
    operator.floordiv.__name__: operator.floordiv,
    operator.truediv.__name__: operator.truediv,
    'protected_div': operator.truediv,
    'protected_pow': operator.pow,
    math.log.__name__: sp.log,
    math.sin.__name__: sp.sin,
    math.cos.__name__: sp.cos,
    math.tan.__name__: sp.tan
}

# Synthetic dataset

For this simple task, we first choose a ground truth function $f$ to generate a dataset $D$. Then, 50 input-output exampels are generated randomly.

In [2]:
def f(x):
    """Ground truth function"""
    return -2 * x ** 2 + 11 * x + 13

In [3]:
n_cases = 100
X = np.random.uniform(-10, 10, size=n_cases)   # random numbers in range [-10, 10)
Y = f(X) + np.random.normal(size=n_cases) # Gaussian noise

# Creating the primitives set
The first step in GEP (or GP as well) is to specify the primitive set, which contains the elementary building blocks to formulate the model. For this problem, we have:
+ function set: the standard arithmetic operators addition (+), subtraction (-), multiplication (*), and division (/).
+ terminal set: only the single input 'x' and random numerical constants (RNC).

NOTE:

- We define a *protected division* to avoid dividing by zero.
- Even there may be multiple RNCs in the model, we only need to call `PrimitiveSet.add_rnc` once.

In [4]:
def protected_pow(x1, x2):
    result = np.power(float(abs(x1)),x2)
    # try:
    #     result = abs(x1)**x2
    # except:
    #     result = 2**20
    # else:
    #     result = abs(x1)**x2
    return np.min([result, 2**30])

def protected_div(x1, x2):
    if abs(x2) < 1e-6:
        return 1
    return x1 / x2


In [5]:
import operator 

pset = gep.PrimitiveSet('Main', input_names=['r','tri','dc','ep','ep2','vpm'])
pset.add_function(operator.add, 2)
pset.add_function(operator.sub, 2)
pset.add_function(operator.mul, 2)
pset.add_function(protected_div, 2)
# pset.add_function(protected_pow, 2)
pset.add_rnc_terminal()

# Create the individual and population
Our objective is to **minimize** the MSE (mean squared error) for data fitting.
## Define the indiviudal class, a subclass of *gep.Chromosome*

In [6]:
from deap import creator, base, tools

creator.create("FitnessMin", base.Fitness, weights=(-1,))  # to minimize the objective (fitness)
creator.create("Individual", gep.Chromosome, fitness=creator.FitnessMin)

## Register the individual and population creation operations
In DEAP, it is recommended to register the operations used in evolution into a *toolbox* to make full use of DEAP functionality. The configuration of individuals in this problem is:
+ head length: 6
+ number of genes in each chromosome: 2
+ RNC array length: 8

Generally, more complicated problems require a larger head length and longer chromosomes formed with more genes. **The most important is that we should use the `GeneDc` class for genes to make use of the GEP-RNC algorithm.**

In [7]:
h = 6 # head length
n_genes = 2   # number of genes in a chromosome
r = 8   # length of the RNC array

In [8]:
toolbox = gep.Toolbox()
toolbox.register('rnc_gen', random.uniform, a=0, b=5)   # each RNC is random within [0, 5]
toolbox.register('gene_gen', gep.GeneDc, pset=pset, head_length=h, rnc_gen=toolbox.rnc_gen, rnc_array_length=r)
toolbox.register('individual', creator.Individual, gene_gen=toolbox.gene_gen, n_genes=n_genes, linker=operator.add)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# compile utility: which translates an individual into an executable function (Lambda)
toolbox.register('compile', gep.compile_, pset=pset)

In [9]:
toolbox.register('select', tools.selTournament, tournsize=3)
# 1. general operators
toolbox.register('mut_uniform', gep.mutate_uniform, pset=pset, ind_pb=0.05, pb=1)
toolbox.register('mut_invert', gep.invert, pb=0.1)
toolbox.register('mut_is_transpose', gep.is_transpose, pb=0.1)
toolbox.register('mut_ris_transpose', gep.ris_transpose, pb=0.1)
toolbox.register('mut_gene_transpose', gep.gene_transpose, pb=0.1)
toolbox.register('cx_1p', gep.crossover_one_point, pb=0.3)
toolbox.register('cx_2p', gep.crossover_two_point, pb=0.2)
toolbox.register('cx_gene', gep.crossover_gene, pb=0.1)
# 2. Dc-specific operators
toolbox.register('mut_dc', gep.mutate_uniform_dc, ind_pb=0.05, pb=1)
toolbox.register('mut_invert_dc', gep.invert_dc, pb=0.1)
toolbox.register('mut_transpose_dc', gep.transpose_dc, pb=0.1)
# for some uniform mutations, we can also assign the ind_pb a string to indicate our expected number of point mutations in an individual
toolbox.register('mut_rnc_array_dc', gep.mutate_rnc_array_dc, rnc_gen=toolbox.rnc_gen, ind_pb='0.5p')
toolbox.pbs['mut_rnc_array_dc'] = 1  # we can also give the probability via the pbs property

In [10]:
stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Define the fitness evaluation function
In DEAP, the single objective optimization problem is just a special case of more general multiobjective ones. Since *geppy* is built on top of DEAP, it conforms to this convention. **Even if the fitness only contains one measure, keep in mind that DEAP stores it as an iterable.** 

Knowing that, you can understand why the evaluation function must return a tuple value (even if it is a 1-tuple). That's also why we set ``weights=(-1,)`` when creating the ``FitnessMax`` class.

In [11]:
import mpse
iteration = 1000 #maximun time iteration
evtime = 2
# size of population and number of generations
n_pop = 100
n_gen = 80

previous_gen = -1
case_list = [None for _ in range(evtime)]

def evaluate(individual, gen):
    """Evalute the fitness of an individual: MAE (mean absolute error)"""
    global previous_gen, case_list
    func = toolbox.compile(individual)
    func_vec = np.vectorize(func)
    itsum = 0
    for i in range(evtime):
        if gen != previous_gen:
            case_list[i] = mpse.gen_case(0.5)
            # case_list[i] = mpse.gen_case(gen/n_gen)
            # print('new gen')
        it = mpse.get_reward(case_list[i],iteration,func_vec)
        itsum = itsum + it
    previous_gen = gen
    # print(itsum)
    return itsum/evtime,

In [12]:
toolbox.register('evaluate', evaluate)

In [13]:
pop = toolbox.population(n=n_pop)
hof = tools.HallOfFame(10)   # only record the best three individuals ever found in all generations

# start evolution
pop, log = gep.gep_simple(pop, toolbox, n_generations=n_gen, n_elites=1,
                          stats=stats, hall_of_fame=hof, verbose=True)

gen	nevals	avg    	std    	min 	max 
0  	100   	1911.71	20.7266	1795	1933
1  	100   	1905.67	30.847 	1788.5	1926.5
2  	100   	1878.26	40.096 	1761.5	1920  
3  	100   	1878.17	44.8943	1777.5	1923  
4  	100   	1867.77	42.4342	1771  	1915.5
5  	100   	1847.4 	49.99  	1757  	1919.5
6  	100   	1844.56	50.7827	1774  	1921  
7  	100   	1829.13	53.12  	1761  	1910.5
8  	100   	1843.86	50.205 	1780.5	1914  
9  	100   	1824.17	57.4581	1757.5	1925  
10 	100   	1843.9 	46.625 	1803.5	1923.5
11 	100   	1811.19	59.4167	1730.5	1921.5
12 	100   	1832.82	35.6921	1793  	1908  
13 	100   	1829.3 	51.7603	1772  	1915.5
14 	100   	1782.01	65.6699	1724  	1920  
15 	100   	1819.23	38.0234	1772.5	1915  
16 	100   	1800.08	57.9675	1755.5	1904.5
17 	100   	1800.15	51.4158	1763.5	1910  
18 	100   	1833.62	43.9045	1799  	1924  
19 	100   	1833.47	30.8902	1806.5	1891  
20 	100   	1811.05	45.8192	1779  	1913  
21 	100   	1826.41	48.8584	1783  	1929.5
22 	100   	1824.02	38.6487	1791.5	1918.5
23 	100   	1773.17	73.24

**Let's check the best individuals ever evolved.**

In [14]:
print(hof[0])

add(
	protected_div(tri, sub(dc, r)),
	mul(sub(protected_div(tri, ep2), protected_div(ep2, dc)), ep2)
)


In [15]:
print('Symplified best individual: ')
# best_ind = hof[0]
for i in range(len(hof)):
    symplified_best = gep.simplify(hof[i], sym_map)
    print(symplified_best)

Symplified best individual: 
tri + tri/(dc - r) - ep2**2/dc
-r*tri - 5 + tri/dc
ep - r*tri/2 + 2*tri
tri*(-5*r + tri*(r + 4))/(r*(r + 4))
r*tri*(ep2 + 1)
dc*r*tri/ep2**2 + ep2 - ep2*tri/ep
dc*r*tri/ep2**2 + tri - ep2*tri/ep
dc*r*tri/ep2**2 + tri - ep2*tri/ep
tri*(dc*r + ep2**2*(tri + 3))/ep2**2
dc*r*tri/ep2**2 + 3*ep2


# *[optional]* Post-processing: simplification and visualization
## Symbolic simplification of the final solution
The original solution seems a little complicated, which may contain many redundancies, for example, `protected_div(x, x)` is just 1. We can perform symbolic simplification of the final result by `geppy.simplify` which depends on `sympy` package.

As we can see from the above simplified expression, the *truth model* has been successfully found. Due to the existence of Gaussian noise, the minimum mean absolute error （MAE) is still not zero even the best individual represents the true model.

## Visualization
If you are interested in the expression tree corresponding to the individual, i.e., the genotype/phenotype system, *geppy* supports tree visualization by the `graph` and the `export_expression_tree` functions:

- `graph` only outputs the nodes and links information to describe the tree topology, with which you can render the tree with tools you like;
- `export_expression_tree` implements tree visualization with data generated by `graph` internally using the `graphviz` package. 

In [16]:
#  # we want use symbol labels instead of words in the tree graph
# rename_labels = {'add': '+', 'sub': '-', 'mul': '*', 'protected_div': '/'}  
# gep.export_expression_tree(best_ind, rename_labels, 'data/numerical_expression_tree.png')

In [17]:
# # show the above image here for convenience
# from IPython.display import Image
# Image(filename='data/numerical_expression_tree.png') 

# Discussion
If only integer constants are involved, then the GEP-RNC algorithm is very effective. If the constants are general real numbers, then more advanced techniques like local search may be more suitable.

Alternatively, the ephemeral numerical constant (ENC) based method can also be adopted here. Please check [ENC for numerical expression inference](./numerical_expression_inference-ENC.ipynb).
